In [365]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.cluster import KMeans
from sklearn import preprocessing, model_selection
import numpy as np
from mainapp.mydata import *
from sklearn.cluster import MeanShift #as ms
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA


In [366]:
 style.use('ggplot')

In [367]:
user = authe.sign_in_with_email_and_password("flaresight.co@outlook.com", "admin123")
uid = user['localId']
token = user['idToken']

In [368]:
 std = db.child("Evaluations").get(token).val()
questionlist=db.child('Questions').get(token).val()
courselist=db.child('Courses').get(token).val()
topiclist=db.child('Topics').get(token).val()

In [369]:
df = pd.DataFrame.from_dict(std, orient='index')
df.reset_index(level=0, inplace = True)
df= df.rename({'index': 'question_atemptid'}, axis=1)
df.head()

,question_atemptid,age,gender,grade,iscorrect,questionid,quizattempt,quizpercent,quiztime,special_care_needed,student_id,subject_of_interest,time_taken,workout_time
0,-LIF44zeF4LWdvNBFxwu,23.0,1,-LIA5tyjgRx5wqDwcjfb,0,-LIAPlNTUTYxvvi_jEuP,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,15.139,afternoon
1,-LIF4Q0hSwGgCqOpTofi,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIAQ2-AyPlYhSM_JQgL,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,77.519,afternoon
2,-LIF4Tv4W0WHEWh3vp0N,23.0,1,-LIA5tyjgRx5wqDwcjfb,0,-LIAQmigkXmiEoFqPP2V,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,11.531,afternoon
3,-LIF4X8hVsanbu6ZpU5_,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIARC5vxksoJzKlQvmS,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,8.171,afternoon
4,-LIF4qQ5L7MHVI-ekP5M,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIARVSWBZ_bGqOmN4CF,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,76.772,afternoon


In [370]:
def getcname(x):
   cid=questionlist[x]['course']
   coursename=courselist[cid]['title']
   return coursename

def gettopicname(x):
   tid=questionlist[x]['topic']
   topicname=topiclist[tid]['title']
   return topicname

In [371]:
df['course']=df['questionid'].map(lambda x:getcname(x))
df['topic']=df['questionid'].map(lambda x:gettopicname(x))

In [372]:
def getcourseframe(course_name):
    x = df[df['course'] == course_name]
    return x

In [373]:
def gettopicframe(topic_name):
    x = df[df['topic'] == topic_name]
    return x

In [374]:
# df = getcourseframe("Grade 1: Astronomy")
df.head()

,question_atemptid,age,gender,grade,iscorrect,questionid,quizattempt,quizpercent,quiztime,special_care_needed,student_id,subject_of_interest,time_taken,workout_time,course,topic
0,-LIF44zeF4LWdvNBFxwu,23.0,1,-LIA5tyjgRx5wqDwcjfb,0,-LIAPlNTUTYxvvi_jEuP,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,15.139,afternoon,Grade 1: Astronomy,First Things First
1,-LIF4Q0hSwGgCqOpTofi,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIAQ2-AyPlYhSM_JQgL,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,77.519,afternoon,Grade 1: Astronomy,First Things First
2,-LIF4Tv4W0WHEWh3vp0N,23.0,1,-LIA5tyjgRx5wqDwcjfb,0,-LIAQmigkXmiEoFqPP2V,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,11.531,afternoon,Grade 1: Astronomy,First Things First
3,-LIF4X8hVsanbu6ZpU5_,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIARC5vxksoJzKlQvmS,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,8.171,afternoon,Grade 1: Astronomy,First Things First
4,-LIF4qQ5L7MHVI-ekP5M,23.0,1,-LIA5tyjgRx5wqDwcjfb,1,-LIARVSWBZ_bGqOmN4CF,-LIF4091axPZFXBYM5G4,60.0,404.478,1,dASdWEa63YgyflDGk6s3egWukEl2,django,76.772,afternoon,Grade 1: Astronomy,First Things First


In [375]:
# df = gettopicframe("Heavenly Bodies And The Universe")

In [376]:
numeric = df[['gender', 'quiztime','special_care_needed', 'iscorrect', 'time_taken','quizpercent', 'age']]
len(numeric)

1280

In [377]:
string = df[['workout_time',]]
df1 = string.apply(lambda s: s.map({k:i for i,k in enumerate(s.unique())}))
df1.head()

,workout_time
0,0
1,0
2,0
3,0
4,0


In [378]:
result = pd.concat([numeric, df1], axis=1, join='inner')
result
result.head()

,gender,quiztime,special_care_needed,iscorrect,time_taken,quizpercent,age,workout_time
0,1,404.478,1,0,15.139,60.0,23.0,0
1,1,404.478,1,1,77.519,60.0,23.0,0
2,1,404.478,1,0,11.531,60.0,23.0,0
3,1,404.478,1,1,8.171,60.0,23.0,0
4,1,404.478,1,1,76.772,60.0,23.0,0


In [379]:
age_mean = result['age'].mean()
result['age'] = result['age'].fillna(age_mean)
X = result
X = X.fillna(value = 0)
X.head()
del X['time_taken']
del X['quiztime']

In [380]:
scaler = StandardScaler()

In [381]:
X_train, X_test = model_selection.train_test_split(X, test_size = 0.2)
kmeans = KMeans(n_clusters = 2)
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit(X_train)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kmeans', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0))])

In [382]:
X_train['cluster'] = pipeline.fit_predict(X_train)
X_train

/home/surazz/PycharmProjects/LMS/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,gender,special_care_needed,iscorrect,quizpercent,age,workout_time,cluster
1034,1,1,1,50.00,23.000000,0,1
687,1,0,1,100.00,23.000000,0,1
1180,1,0,0,20.00,23.000000,0,0
1136,1,0,1,56.25,23.000000,0,1
792,1,0,0,33.33,23.000000,1,0
636,1,0,1,80.00,23.000000,0,1
514,0,0,0,6.25,23.000000,0,0
266,1,1,0,26.67,23.000000,0,0
120,0,0,0,14.29,23.000000,0,0
38,1,1,1,60.00,23.000000,0,1


In [383]:
y=X_train.loc[(X_train['quizpercent']>=90) & (X_train['iscorrect']==1)]
x=y.iloc[0]
x
clustervalue=x['cluster']
def namingcluster(x):
    if x==clustervalue:
        return "strong"
    else:
        return "weak"
X_train['cluster']=X_train['cluster'].map(lambda x:namingcluster(x))
X_train



/home/surazz/PycharmProjects/LMS/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,gender,special_care_needed,iscorrect,quizpercent,age,workout_time,cluster
1034,1,1,1,50.00,23.000000,0,strong
687,1,0,1,100.00,23.000000,0,strong
1180,1,0,0,20.00,23.000000,0,weak
1136,1,0,1,56.25,23.000000,0,strong
792,1,0,0,33.33,23.000000,1,weak
636,1,0,1,80.00,23.000000,0,strong
514,0,0,0,6.25,23.000000,0,weak
266,1,1,0,26.67,23.000000,0,weak
120,0,0,0,14.29,23.000000,0,weak
38,1,1,1,60.00,23.000000,0,strong


In [384]:
labels = pipeline.predict(X_test)
# X_train['cluster'] = kmeans.fit_predict(X_train)
X_test['cluster'] = labels
X_test


/home/surazz/PycharmProjects/LMS/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,gender,special_care_needed,iscorrect,quizpercent,age,workout_time,cluster
0,1,1,0,60.00,23.000000,0,0
812,1,0,1,33.33,23.000000,1,1
527,1,1,0,42.11,23.000000,0,0
42,1,1,1,60.00,23.000000,0,1
277,0,0,1,46.67,23.000000,0,0
978,1,0,1,93.75,23.000000,1,1
446,0,0,1,40.00,23.000000,0,0
834,1,0,1,76.47,23.000000,0,1
85,0,0,0,50.00,23.000000,0,0
1001,1,0,0,66.67,23.000000,1,1


In [385]:
df2 = pd.DataFrame({'labels': labels, 'students': X_test['student_id']})
ct = pd.crosstab(df2['students'], df2['labels'])
ct

KeyError: 'student_id'